# Model Inference Comparison & Validation Script

In [ ]:
import csv, json, glob, os, pandas as pd, objdict, ast

compareDF = pd.DataFrame(columns=['VideoId'])
modelCount = -1

This Model Comparison Script takes in a json file directory where each json file has been normalized in the format below:

``` json
{
  "VideoId": "some-identifier.wav",
  "LabelData": [
    {
      "label_0" "427",
      "labelConf_0": "0.4131"
    },
    {
      "label_1": "213",
      "labelConf_1": ".3121"
    },
    {
      "label_2": "0",
      "labelConf_2": ".2421"
    }
  ]
}
```

In [ ]:
#Pass a directory of formatted Inference Json files
for jsonfile in glob.glob('NormalizedInferenceScores/*json'):
    jsonDF = pd.read_json(jsonfile)
    modelCount += 1
    print(str(jsonfile) +" has been indexed at: Model_"+ str(modelCount) )
    #Iterate through the rows and reformat based on key/value pairs
    for index, row in jsonDF.iterrows():
        
        #Check if VideoId currently exists in the master compare dataframe otherwise add it
        if(compareDF['VideoId'].str.contains(str(row['VideoId'])).any()):
            if(index == 0):
                print("Audio Inference file contained preexisting VideoIds, Updating compare dataframe with values from Model_" +str(modelCount))
            jsonRow = row['Label_Data']
            convertedRow = pd.DataFrame(columns=['VideoId'])
            indexWrite = compareDF[compareDF['VideoId']==row['VideoId']].index.values.astype(int)[0]
            #For all labels and confidence pairs, add as new column in the data frame
            for field in jsonRow:
                iterativeLabel = "Model"+str(modelCount)+"_"+field
                if (index == 0):
                    compareDF = compareDF.reindex(columns=[*compareDF.columns.tolist(), iterativeLabel], fill_value=0.0)
                compareDF.loc[indexWrite, iterativeLabel] = jsonRow[field]

        #Add VideoId to master compare dataframe with values   
        else:
            if (index == 0):
                print("New Audio Inference file detected with new videos, Adding new VideoIDs from Model_" +str(modelCount))
            jsonRow = row['Label_Data']
            convertedRow = pd.DataFrame(columns=['VideoId'])
            #For all labels and confidence pairs, add as new column in the data frame
            for field in jsonRow:
                #Create Columns for new model
                iterativeLabel = "Model"+str(modelCount)+"_"+field
                if (index == 0):
                    compareDF = compareDF.reindex(columns=[*compareDF.columns.tolist(), iterativeLabel], fill_value=0.0)  
                convertedRow.loc[0, iterativeLabel] = jsonRow[field]
            # Add Temp row to Compare Dataframe    
            convertedRow.loc[0,'VideoId'] = row['VideoId']
            compareDF = compareDF.append(convertedRow, ignore_index=True)
    

In [ ]:
compareDF

## Outstanding things to do:
- Reindex Columns so they are in a sequential format. I.E. (Model0_Label0 | Model0_Label_0_Conf)
- Fix sorting warning:
```
FutureWarning: Sorting because non-concatenation axis is not aligned. A future version of pandas will change to not sort by default.
To accept the future behavior, pass 'sort=False'.
To retain the current behavior and silence the warning, pass 'sort=True'.
sort=sort)
```
- Move model index to directory base
- Persisted Dataframes
- Validation Cells for Gunshot (Inputs for Validation)
- Migrate to a script